In [1]:
import findspark
findspark.init()

# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my app").master("local").getOrCreate() # spark deamon 생성 # 이전에는 spark.sparkContext

# get context from the session
sc = spark.sparkContext

In [2]:
strrdd =sc.parallelize ([
    '{ \"name\" : \"a\", \"id\": \"1\"}',
    '{ \"name\" : \"b\", \"id\": \"2\"}'
])
strrdd.collect()

['{ "name" : "a", "id": "1"}', '{ "name" : "b", "id": "2"}']

In [3]:
# jsonrdd parsing
jsonrdd = spark.read.json(strrdd)
jsonrdd.show()

+---+----+
| id|name|
+---+----+
|  1|   a|
|  2|   b|
+---+----+



In [4]:
# 
jsonrdd.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



## Schema 새롭게 정의하고 싶다

### createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True)
***Creates a DataFrame from an RDD, a list*** or a pandas.DataFrame.

When schema is a list of column names, the type of each column will be inferred from data.

When schema is None, *it will try to infer the schema (column names and types) from data*, which should be an RDD of either Row, namedtuple, or dict.

When schema is pyspark.sql.types.DataType or a datatype string, it must match the real data, or an exception will be thrown at runtime. If the given schema is not pyspark.sql.types.StructType, it will be wrapped into a pyspark.sql.types.StructType as its only field, and the field name will be “value”. Each record will also be wrapped into a tuple, which can be converted to row later.

If schema inference is needed, samplingRatio is used to determined the ratio of rows used for schema inference. The first row will be used if samplingRatio is None.

Parameters
* data – an RDD of any kind of SQL data representation (e.g. row, tuple, int, boolean, etc.), list, or pandas.DataFrame.
* schema – a pyspark.sql.types.DataType or a datatype string or a list of column names, default is None.
* samplingRatio – the sample ratio of rows used for inferring
* verifySchema – verify data types of every row against schema.

In [5]:
# import 필요
from pyspark.sql.types import *
schema = StructType([
    StructField("id", LongType(), False),
    StructField("name", StringType(), False)
])

#### df변환 처음 시도

In [6]:
df  = spark.createDataFrame(jsonrdd, schema)

TypeError: data is already a DataFrame

In [7]:
# jsonrdd.rdd 변환
df  = spark.createDataFrame(jsonrdd.rdd, schema)
df.show() # error 해석할수 않음

Py4JJavaError: An error occurred while calling o61.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/pyspark/pkg/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/pyspark/pkg/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/pyspark/pkg/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/pyspark/pkg/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "/home/pyspark/pkg/spark-2.4.5-bin-

In [8]:
# jsonrdd의 rdd는 제대로 있다 ==> 데이터타입 변환 시도
jsonrdd.rdd.collect()

[Row(id='1', name='a'), Row(id='2', name='b')]

In [9]:
from pyspark.sql.types import *
schema = StructType([
    StructField("id", LongType(), False),
    StructField("name", StringType(), False)
])
# 데이터타입 변환(casting)
# simplerdd = jsonrdd.rdd.map(lambda x: (int(x[0]), int(x[1]) ) ) # int로 잘못 casting
simplerdd = jsonrdd.rdd.map(lambda x: (int(x[0]), x[1]  ) )
simplerdd.collect()                            

[(1, 'a'), (2, 'b')]

In [10]:
# simplerdd  --> df 변환
df = spark.createDataFrame(simplerdd, schema)
df.show()

+---+----+
| id|name|
+---+----+
|  1|   a|
|  2|   b|
+---+----+



In [11]:
df.printSchema() # string nullable = True인데 계속 false됨

root
 |-- id: long (nullable = false)
 |-- name: string (nullable = false)



## 파일을 직접 df로 불러읽자

### json(path, schema=None, ...)

Loads JSON files and returns the results as a DataFrame.

JSON Lines (newline-delimited JSON) is supported by default. For JSON (one record per file), set the multiLine parameter to true.

If the schema parameter is not specified, this function goes through the input once to determine the input schema.

Parameters
* path – string represents path to the JSON dataset, or a list of paths, or RDD of Strings storing JSON objects.
* schema – an optional pyspark.sql.types.StructType for the input schema or a DDL-formatted string (For example col0 INT, col1 DOUBLE).

In [12]:
# df = spark.read.format("json").load("2015-summary.json") # 동일한 결과 나옴
df = spark.read.json("2015-summary.json")
df.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows



### 실습과제 tip

In [13]:
# 데이터를 보면 tap구분되어 있다(userid, movieid, rating,timestamp)
udata = sc.textFile("u.data").map(lambda x: [int (a) for a in x.split('\t')]) # int a error # int (a)
udata.take(5)

[[196, 242, 3, 881250949],
 [186, 302, 3, 891717742],
 [22, 377, 1, 878887116],
 [244, 51, 2, 880606923],
 [166, 346, 1, 886397596]]

In [14]:
# df 변환
schema = StructType([
    StructField("uid", LongType(), False),
    StructField("mid", LongType(), False),
    StructField("rate", LongType(), False),
    StructField("timestamp", LongType(), False)
])
udata = spark.createDataFrame(udata, schema)
udata.show(5)

+---+---+----+---------+
|uid|mid|rate|timestamp|
+---+---+----+---------+
|196|242|   3|881250949|
|186|302|   3|891717742|
| 22|377|   1|878887116|
|244| 51|   2|880606923|
|166|346|   1|886397596|
+---+---+----+---------+
only showing top 5 rows



### df 변환 다른 방법

In [15]:
udata2 = spark.read.csv("u.data", schema) # sep를 tap지정하지 않으면 모두 null
udata2.show(3)

+----+----+----+---------+
| uid| mid|rate|timestamp|
+----+----+----+---------+
|null|null|null|     null|
|null|null|null|     null|
|null|null|null|     null|
+----+----+----+---------+
only showing top 3 rows



In [16]:
schema = StructType([
    StructField("uid", LongType(), False),
    StructField("mid", LongType(), False),
    StructField("rate", LongType(), False),
    StructField("timestamp", LongType(), False)
])
udata2 = spark.read.format("csv").option("sep", "\t" ).schema(schema).load("u.data")
udata2.show(3)

+---+---+----+---------+
|uid|mid|rate|timestamp|
+---+---+----+---------+
|196|242|   3|881250949|
|186|302|   3|891717742|
| 22|377|   1|878887116|
+---+---+----+---------+
only showing top 3 rows



In [17]:
udata2.printSchema()

root
 |-- uid: long (nullable = true)
 |-- mid: long (nullable = true)
 |-- rate: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [18]:
schema = StructType([
    StructField("mid", LongType(), False),
    StructField("title", StringType(), False)
])
uitem = spark.read.format("csv").option("sep", "|").schema(schema).load("u.item")
uitem.printSchema() # true 변경되는 이유는 잘 모르겠음

root
 |-- mid: long (nullable = true)
 |-- title: string (nullable = true)



## sql처름 질의할 수 있돌록 임시 tempView만들기 ==> createOrReplaceTempView

### createOrReplaceTempView(name)
Creates or replaces a local temporary view with this DataFrame.

***It creates (or replaces if that view name already exists) a lazily evaluated "view" that you can then use like a hive table in Spark SQL.***

It does not persist to memory unless you cache the dataset that underpins the view.
The lifetime of this temporary table is tied to the SparkSession that was used to create this DataFrame.

In [19]:
# from pyspark.sql.functions import col
udata.createOrReplaceTempView("udata")
spark.sql("select * from udata where uid =22 limit 5").show()

+---+---+----+---------+
|uid|mid|rate|timestamp|
+---+---+----+---------+
| 22|377|   1|878887116|
| 22|376|   3|878887112|
| 22|128|   5|878887983|
| 22| 80|   4|878887227|
| 22|241|   3|878888025|
+---+---+----+---------+



### sql에서 사용하기 쉽게 

### select(*cols)
Projects a set of expressions and returns a new DataFrame.

Parameters
* cols – list of column names (string) or expressions (Column). If one of the column names is ‘*’, that column is expanded to include all columns in the current DataFrame.

In [20]:
from pyspark.sql.functions import col
udata.select(col("uid"), col("rate")).show(5)

+---+----+
|uid|rate|
+---+----+
|196|   3|
|186|   3|
| 22|   1|
|244|   2|
|166|   1|
+---+----+
only showing top 5 rows



In [22]:
from pyspark.sql.functions import * # lit import 필요
udata.select(col("uid"), lit(1).alias("one")).show(5)

+---+---+
|uid|one|
+---+---+
|196|  1|
|186|  1|
| 22|  1|
|244|  1|
|166|  1|
+---+---+
only showing top 5 rows



### selectExpr(*expr)
Projects a set of SQL expressions and returns a new DataFrame.

This is a variant of select() that accepts SQL expressions. 

In [23]:
udata.selectExpr("uid", "1 as one").show(5)

+---+---+
|uid|one|
+---+---+
|196|  1|
|186|  1|
| 22|  1|
|244|  1|
|166|  1|
+---+---+
only showing top 5 rows

